#### pre processing

In [ ]:
# _____________ IMPORT
#region
from calendar import c
from ntpath import join
from re import A
from tkinter.tix import DirSelectBox
from zlib import DEF_BUF_SIZE
import pandas as pd
import seaborn as sns # added to OGD_HMM
import numpy as np
import matplotlib.pyplot as plt
import shutil
from glob import glob
from pathlib import Path
# from os import path
import os
# import glob
from IPython.display import display
import math

#### action analysis

In [ ]:
ogd_data = pd.read_csv("Data\gaze_input_tobii_and_ogd\group_difficult\participant01_trial01_ogd.txt", sep='\t')
# drop columns with any NaN values (don't know why they occur sometimes?)
ogd_data.dropna(inplace=True)
ogd_data.reset_index(drop=True, inplace=True) 
            
# initialize the action df
df_actions = pd.DataFrame()
df_actions['Action'] = ogd_data['Action']
df_actions['start_time'] = ogd_data['start_time']
df_actions['end_time'] = ogd_data['end_time']

# add column with action as numbers to simplify finding changes
all_actions = ogd_data['Action'].unique().tolist()
dict_actions = {k: v for v, k in enumerate(all_actions)}
df_actions['Action Numbers'] = (df_actions['Action']).map(dict_actions)

In [ ]:
# create a variable called 'change' that tells whether the event changed (calculates the difference to previous row)
df_actions['change'] = df_actions['Action Numbers'].diff()
# set change to 0 in first column
#df_actions['change'][0] = 1

In [ ]:
# remove rows where the event did not change:
df_actions = df_actions.loc[df_actions['change'] !=0 ,:]


In [ ]:
# assign continous start and end times -> end_time of action 1 = start_time of action 2
# to prevent gaps for further analysis of fixations and saccades
df_actions['start_time_action'] = float(0*len(df_actions))
if df_actions['change'][32] != 0:
    df_actions['start_time_action'][32] = df_actions['end_time'][32-1]

In [ ]:
# add start time of action
df_actions['end_time_action'] = df_actions['start_time'].shift(-1)
df_actions['start_time_action'] = df_actions['start_time']

# add time of last action
df_actions['end_time_action'].iloc[len(df_actions)-1] = ogd_data['end_time'].iloc[len(ogd_data)-1]
df_actions


In [ ]:

# fix first column (starts when very first action in first fixation starts)
df_actions['start_time_action'][0] = ogd_data['start_time'][0]

In [ ]:
df_actions['start_time_event'] = df_actions['end_time'].shift(1)
df_actions['end_time_event'] = df_actions['end_time']

#### all in one

In [8]:
import pandas as pd

ogd_data = pd.read_csv("Data\gaze_input_tobii_and_ogd\group_difficult\participant01_trial01_ogd.txt", sep='\t')
fixationdata = pd.read_csv("Data\gaze_input_tobii_and_ogd\group_difficult\participant01_trial01_fixations.txt", sep='\t')
saccadedata = pd.read_csv("Data\gaze_input_tobii_and_ogd\group_difficult\participant01_trial01_saccades.txt", sep='\t')

# drop columns with any NaN values (don't know why they occur sometimes?)
ogd_data.dropna(inplace=True)
ogd_data.reset_index(drop=True, inplace=True) 

# turn [s] into [ms] for simplicity and change it back for output again (?)
ogd_data['start_time'] = ogd_data['start_time']*1000
ogd_data['end_time'] = ogd_data['end_time'] * 1000

# rename last col to 'action'
ogd_data.rename(columns={ogd_data.columns[-1]: 'action'}, inplace=True)

# initialize the action df
df_actions = pd.DataFrame()
df_actions['action'] = ogd_data['action']
df_actions['start_time'] = ogd_data['start_time']
df_actions['end_time'] = ogd_data['end_time']

# add column with action as numbers to simplify finding changes
all_actions = ogd_data['action'].unique().tolist()
dict_actions = {k: v for v, k in enumerate(all_actions)}
df_actions['Action Numbers'] = (df_actions['action']).map(dict_actions)


# create a variable called 'change' that tells whether the event changed (calculates the difference to previous row)
df_actions['change'] = df_actions['Action Numbers'].diff()

# remove rows where the event did not change:
df_actions = df_actions.loc[df_actions['change'] !=0 ,:]

# 
df_actions['start_time_action'] = df_actions['start_time']
df_actions['end_time_action'] = df_actions['start_time'].shift(-1)


# add time of first and last action
# end of trial time
end_time_seconds = float(fixationdata['Event End Trial Time [ms]'].iloc[len(fixationdata)-1])
df_actions['end_time_action'].iloc[len(df_actions)-1] = end_time_seconds
# start of trial time
df_actions['start_time_action'][0] = float(0)

df_actions.reset_index(inplace=True)
df_actions = df_actions.rename(columns = {'index':'action_change_index'})

df_actions


c:\Users\renee\anaconda3\envs\OGD_HMM\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\Users\renee\anaconda3\envs\OGD_HMM\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,action_change_index,action,start_time,end_time,Action Numbers,change,start_time_action,end_time_action
0,0,Cap Off,850.0,950.0,0,NaN,0.0,4787.0
1,32,Apply Tip,4787.0,4887.0,1,1.0,4787.0,13232.0
2,109,Setting Units,13232.0,13302.0,2,1.0,13232.0,15981.0
3,133,Priming,15981.0,16081.0,3,1.0,15981.0,20438.0
4,173,Setting Units,20438.0,20508.0,2,-1.0,20438.0,21697.0
5,182,Priming,21697.0,21797.0,3,1.0,21697.0,29623.0
6,246,Setting Units,29623.0,29723.0,2,-1.0,29623.0,32801.0
7,274,Injection,32801.0,32901.0,4,2.0,32801.0,53379.0
8,455,Remove Tip,53379.0,53479.0,5,1.0,53379.0,61235.0
9,522,Cap On,61235.0,61335.0,6,1.0,61235.0,69200.0


In [10]:
df_actions

,action_change_index,action,start_time,end_time,Action Numbers,change,start_time_action,end_time_action
0,0,Cap Off,850.0,950.0,0,NaN,0.0,4787.0
1,32,Apply Tip,4787.0,4887.0,1,1.0,4787.0,13232.0
2,109,Setting Units,13232.0,13302.0,2,1.0,13232.0,15981.0
3,133,Priming,15981.0,16081.0,3,1.0,15981.0,20438.0
4,173,Setting Units,20438.0,20508.0,2,-1.0,20438.0,21697.0
5,182,Priming,21697.0,21797.0,3,1.0,21697.0,29623.0
6,246,Setting Units,29623.0,29723.0,2,-1.0,29623.0,32801.0
7,274,Injection,32801.0,32901.0,4,2.0,32801.0,53379.0
8,455,Remove Tip,53379.0,53479.0,5,1.0,53379.0,61235.0
9,522,Cap On,61235.0,61335.0,6,1.0,61235.0,69.2


#### separate fixationdata file

In [26]:
fixationdata

,Event Start Trial Time [ms],Event End Trial Time [ms],Event Duration [ms],Visual Intake Position X [px],Visual Intake Position Y [px],action
0,0.0,195.0,195,829.0,573.0,Cap Off
1,320.0,500.0,180,855.0,535.0,Cap Off
2,580.0,700.0,120,809.0,546.0,Cap Off
3,720.0,810.0,90,807.0,559.0,Cap Off
4,850.0,1100.0,250,916.0,553.0,Cap Off
...,...,...,...,...,...,...
175,67981.0,68261.0,280,1086.0,299.0,Apply Tip
176,68281.0,68401.0,120,1080.0,313.0,Apply Tip
177,68420.0,68680.0,260,1079.0,323.0,Apply Tip
178,68700.0,68890.0,190,1092.0,333.0,Apply Tip


In [9]:
action_counter=0
action_list = []
for row in range(len(fixationdata)):
    action_list.append(df_actions['action'][action_counter])
    # if last row
    if row == len(fixationdata)-1:
        exit
    # if change 
    elif fixationdata['Event End Trial Time [ms]'][row+1] > df_actions['end_time_action'][action_counter]:
        action_counter = action_counter + 1
fixationdata['action'] = action_list

22
43
51
61
66
78
89
131
151


In [7]:
fixationdata

,Event Start Trial Time [ms],Event End Trial Time [ms],Event Duration [ms],Visual Intake Position X [px],Visual Intake Position Y [px],action
0,0.0,195.0,195,829.0,573.0,Cap Off
1,320.0,500.0,180,855.0,535.0,Cap Off
2,580.0,700.0,120,809.0,546.0,Cap Off
3,720.0,810.0,90,807.0,559.0,Cap Off
4,850.0,1100.0,250,916.0,553.0,Cap Off
...,...,...,...,...,...,...
175,67981.0,68261.0,280,1086.0,299.0,Cap On
176,68281.0,68401.0,120,1080.0,313.0,Cap On
177,68420.0,68680.0,260,1079.0,323.0,Cap On
178,68700.0,68890.0,190,1092.0,333.0,Cap On


In [10]:
action_counter=0
action_list = []
for row in range(len(saccadedata)):
    action_list.append(df_actions['action'][action_counter])
    # if last row
    if row == len(saccadedata)-1:
        exit
    # if change 
    elif saccadedata['Event End Trial Time [ms]'][row+1] > df_actions['end_time_action'][action_counter]:
        print(row)
        action_counter = action_counter + 1
saccadedata['action'] = action_list

7
26
30
37
43
58
62
101
121


In [11]:
saccadedata

,Event Start Trial Time [ms],Event End Trial Time [ms],Event Duration [ms],action
0,1709.002,1729.002,20,Cap Off
1,1718.993,1738.993,20,Cap Off
2,2558.508,2578.508,20,Cap Off
3,2568.526,2588.526,20,Cap Off
4,2798.396,2828.396,30,Cap Off
...,...,...,...,...
158,67081.246,67111.246,30,Cap On
159,67221.158,67261.158,40,Cap On
160,67231.150,67271.150,40,Cap On
161,67241.146,67281.146,40,Cap On


#### separate ogd_data into new dataframes and calculate ooi_metrics

In [ ]:
# extract actions
all_actions = ogd_data['action'].unique().tolist()

In [ ]:
all_actions

In [ ]:
df_new = ogd_data[ogd_data['action'] == 'Cap Off']

In [ ]:
df_new

In [ ]:
for i in range(1,len(df_actions)):
    df_new = ogd_data[df_actions.index[i-1]:df_actions.index[i]]

In [ ]:
df_new

In [ ]:
cut_off_rows = []
i=0
for row in range(1,len(ogd_data)-1):
    print('row:', row)
    print('i:', i)
    if ogd_data['end_time'][row] > df_actions['end_time_action'][i]:
        cut_off_rows.append(row-1)
        print('i:', i)
        i=i+1


In [ ]:
cut_off_rows

In [ ]:
ogd_data.iloc[0:5]